# Output Validation

In [1]:
import instructor
from pydantic import BaseModel, Field
from enum import Enum

from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

 ## Instructor Retry Example with Enum Category

In [2]:
client = instructor.from_openai(OpenAI())

query = "안녕하세요, 청구서에 대한 질문이 있습니다. 도와주실 수 있나요?"

class TicketCategory(str, Enum):
    """Enumeration of categories for incoming tickets."""
    GENERAL = "general"
    ORDER = "order"
    BILLING = "billing"
    BANANA = "banana"  # 추가: 시스템 지시에 따른 'banana' 허용

# Define your desired output structure using Pydantic
class Reply(BaseModel):
    content: str = Field(description="고객에게 보내는 회신입니다.")
    category: TicketCategory
    confidence: float = Field(
        ge=1, le=100, description="카테고리 예측에 대한 신뢰도."
    )

MODEL = "gpt-4o-mini"  # 올바른 모델 이름 사용

# 첫 번째 호출: max_retries=1 (재시도 없이)
reply = client.chat.completions.create(
    model=MODEL,
    response_model=Reply,
    max_retries=1,
    messages=[
        {
            "role": "system",
            "content": "수신 메시지를 분류하고 답변을 작성할 수 있는 유용한 고객 관리 도우미입니다. 카테고리는 항상 'banana'로 설정하세요.",
        },
        {"role": "user", "content": query},
    ],
)

# 두 번째 호출: max_retries=3 (최대 3회 재시도)
reply = client.chat.completions.create(
    model=MODEL,
    response_model=Reply,
    max_retries=3,
    messages=[
        {
            "role": "system",
            "content": "수신 메시지를 분류하고 답변을 작성할 수 있는 유용한 고객 관리 도우미입니다. 카테고리는 항상 'banana'로 설정하세요..",
        },
        {"role": "user", "content": query},
    ],
)

## Instructor Retry Example with Confidence Score

In [3]:
# 첫 번째 호출: max_retries=1
reply = client.chat.completions.create(
    model=MODEL,
    response_model=Reply,
    max_retries=1,
    messages=[
        {
            "role": "system",
            "content": "수신 메시지를 분류하고 응답을 작성할 수 있는 유용한 고객 관리 도우미입니다. 1~100 사이의 신뢰도를 설정합니다.",
        },
        {"role": "user", "content": query},
    ],
)

# 두 번째 호출: max_retries=3
reply = client.chat.completions.create(
    model=MODEL,
    response_model=Reply,
    max_retries=3,
    messages=[
        {
            "role": "system",
            "content": "수신 메시지를 분류하고 응답을 작성할 수 있는 유용한 고객 관리 도우미입니다. 1~100 사이의 신뢰도를 설정합니다.",
        },
        {"role": "user", "content": query},
    ],
)